#### Mount Azure Data Lake Containers for the project


In [0]:
def mount_adls(storage_account_name, container_name):
    # Get secrets from Key vault
    client_id = dbutils.secrets.get(scope = "formula1-scope", key = "formula1-app-client-id")
    tenant_id =  dbutils.secrets.get(scope = "formula1-scope", key = "formula1-app-tenant-id")
    client_secret = dbutils.secrets.get(scope = "formula1-scope", key = "formula1-app-client-secret")

    #Set spark configuratios

    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    #unmount if already mounted
    
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")
    
    #mount the storage

    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_account_name}/{container_name}",
        extra_configs = configs)
    
    display(dbutils.fs.mounts())

In [0]:
mount_adls("f1dlportfolio", "raw")

In [0]:
mount_adls("f1dlportfolio", "processed")

In [0]:
mount_adls("f1dlportfolio", "presentation")

#### Mount Raw Container first